In [ ]:
import os

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import pickle
import joblib
import json
from datetime import datetime

In [ ]:
scaler_1h = joblib.load('../src/our_models/scaler_1hour.pkl')
scaler_2h = joblib.load('../src/our_models/scaler_2hour.pkl')
scaler_all = joblib.load('../src/our_models/scaler_all.pkl')

In [ ]:
with open('../src/our_models/3_Xgboost_1hour.pkl', 'rb') as f:
    model_1h = pickle.load(f)
with open('../src/our_models/3_Xgboost_2hour.pkl', 'rb') as f:
    model_2h = pickle.load(f)
with open('../src/our_models/3_Xgboost_all.pkl', 'rb') as f:
    model_all = pickle.load(f)

In [ ]:
df = pd.read_csv("../data/everyhour_predict/test.csv",sep=";")

In [ ]:
df

In [ ]:
#X_new = df[df['region_id'] == 21]
#X_1 = X_new.iloc[[0]]
#X_2 = X_new.iloc[[1]]
#X_all = X_new.iloc[2:]
#X_all = X_all.drop("status", axis=1)

In [ ]:
#X_1_scaled = scaler_1h.transform(X_1)
#X_2_scaled = scaler_2h.transform(X_2)
#X_all_scaled = scaler_all.transform(X_all)

In [ ]:
#y_pred_1 = model_1h.predict(X_1_scaled)

In [ ]:
#y_pred_1

In [ ]:
#y_pred_2 = model_2h.predict(X_2_scaled)

In [ ]:
#y_pred_2

In [ ]:
#y_pred_all = model_all.predict(X_all_scaled)
#y_pred_all

In [ ]:
#df

In [ ]:
#scaler_1h

In [ ]:
#prediction = np.concatenate([y_pred_1, y_pred_2, y_pred_all])
#hours = [f"{h:02d}:00" for h in range(24)]
#forecast = {
#    hour: True if status == 1 else False
#    for hour, status in zip(hours, prediction)
#}

In [ ]:
#forecast

In [ ]:
region_map = {
    1: "Kyiv",
    2: "Vinnytsia",
    3: "Lutsk",
    4: "Dnipro",
    5: "Donetsk",
    6: "Zhytomyr",
    7: "Uzhgorod",
    8: "Zaporozhye",
    9: "Ivano-Frankivsk",
    10: "Kyiv region",
    11: "Kropyvnytskyi",
    13: "Lviv",
    14: "Mykolaiv",
    15: "Odesa",
    16: "Poltava",
    17: "Rivne",
    18: "Sumy",
    19: "Ternopil",
    20: "Kharkiv",
    21: "Kherson",
    22: "Khmelnytskyi",
    23: "Cherkasy",
    24: "Chernivtsi",
    25: "Chernihiv"
}
hours = [f"{h:02d}:00" for h in range(24)]
regions_forecast = {}

In [ ]:
for region_id, region_name in region_map.items():
    X_new = df[df['region_id'] == region_id]
    X_1 = X_new.iloc[[0]]
    X_2 = X_new.iloc[[1]]
    X_all = X_new.iloc[2:]
    X_all = X_all.drop("status", axis=1)
    X_1_scaled = scaler_1h.transform(X_1)
    X_2_scaled = scaler_2h.transform(X_2)
    X_all_scaled = scaler_all.transform(X_all)
    y_pred_1 = model_1h.predict(X_1_scaled)
    y_pred_2 = model_2h.predict(X_2_scaled)
    y_pred_all = model_all.predict(X_all_scaled)
    prediction = np.concatenate([y_pred_1, y_pred_2, y_pred_all])
    forecast = {
        hour: True if status == 1 else False
        for hour, status in zip(hours, prediction)
    }
    regions_forecast[region_name] = forecast
result = {"regions_forecast": regions_forecast}

In [ ]:
result

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data/forecasts')
file_path = os.path.join(data_dir, "forecast.json")
os.makedirs(data_dir, exist_ok=True)
with open(file_path, "w") as f:
    json.dump(result, f)